In [220]:
import os

In [221]:
from google_api import google_api

os.environ['GOOGLE_API_KEY'] = google_api

In [197]:
#pip install psycopg2 -qU

In [198]:
#pip install langchain_experimental -qU

In [199]:
#pip install -q -U langchain

In [200]:
#pip install langchain-google-genai -qU

In [201]:
#pip show langchain

In [202]:
#pip show pydantic

In [203]:
#pip uninstall pydantic -y

In [204]:
#pip install pydantic==2.9.2

In [205]:
#pip show pydantic

In [239]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [240]:
from langchain_experimental.sql import SQLDatabaseChain
from langchain.sql_database import SQLDatabase

In [241]:
db_user = "root"
db_password = "Root123*"
db_host = "localhost"
db_name = "atliq_tshirts"

In [242]:
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
21	1	10.00
22	2	15.00
23	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	st

In [243]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True, top_k=3)

In [246]:
qns1 = db_chain.invoke("How many t-shirts do we have left for Nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for Nike in extra small size and white color?
Question: How many t-shirts do we have left for Nike in extra small size and white color?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White' LIMIT 1;
SQLResult: 
SQLResult: Empty set (0 rows)
Answer: We have 0 t-shirts left for Nike in extra small size and white color.
> Finished chain.


In [261]:
print(qns1)

{'query': 'How many t-shirts do we have left for Nike in extra small size and white color?', 'result': 'SQLResult: Empty set (0 rows)\nAnswer: We have 0 t-shirts left for Nike in extra small size and white color.'}


In [8]:
# PROMPT = """
#     DO NOT USE pre-amble OR ```sql``` CHARACTERS in the SQL query
#     Given the following question, create a corresponding SQL query that retrieves the answer from the database.

#     Question: {question}

#     SQL Query:

#     Ensure the SQL query is structured to retrieve relevant data from the database according to the question.
#     Execute the SQL query
# """

In [9]:
# query = "How many t-shirts do we have left for Nike in extra small size and black color?"

In [10]:
# qns1 = db_chain.invoke(PROMPT.format(question=query))



> Entering new SQLDatabaseChain chain...

    DO NOT USE pre-amble OR ```sql``` CHARACTERS in the SQL query
    Given the following question, create a corresponding SQL query that retrieves the answer from the database.

    Question: How many t-shirts do we have left for Nike in extra small size and black color?

    SQL Query:

    Ensure the SQL query is structured to retrieve relevant data from the database according to the question.
    Execute the SQL query

Question: How many t-shirts do we have left for Nike in extra small size and black color?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'Black' LIMIT 1
SQLResult: [(92,)]
Answer: We have 92 Nike t-shirts left in extra small size and black color.
> Finished chain.


In [258]:
#qns2 = db_chain.invoke("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
Question: How much is the price of the inventory for all small size t-shirts?
SQLQuery: SELECT SUM(`price` * `stock_quantity`) AS `Total Inventory Price` FROM `t_shirts` WHERE `size` = 'S' LIMIT 1;
SQLResult: [(Decimal('19487'),)]
Answer: The total inventory price for all small size t-shirts is 2254.
> Finished chain.


In [249]:
sql_code = """ Generate SQL query without pre-amble or ```sql``` characters :
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from atliq_shirts.t_shirts where brand = 'Levi'
group by t_shirt_id) a left join atliq_shirts.discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.invoke(sql_code)



> Entering new SQLDatabaseChain chain...
 Generate SQL query without pre-amble or ```sql``` characters :
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from atliq_shirts.t_shirts where brand = 'Levi'
group by t_shirt_id) a left join atliq_shirts.discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery: SELECT SUM(`a`.`total_amount` * ((100 - COALESCE(`discounts`.`pct_discount`, 0)) / 100)) AS `total_revenue` FROM (SELECT SUM(`price` * `stock_quantity`) AS `total_amount`, `t_shirt_id` FROM `t_shirts` WHERE `brand` = 'Levi' GROUP BY `t_shirt_id`) AS `a` LEFT JOIN `discounts` ON `a`.`t_shirt_id` = `discounts`.`t_shirt_id`
SQLResult: [(Decimal('21239.550000'),)]
Answer:21239.55
> Finished chain.


In [259]:
qns4 = db_chain.invoke("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
Question: SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLResult: [(Decimal('21723'),)]
Answer: 21723
> Finished chain.


In [260]:
#qns5 = db_chain.invoke("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White' LIMIT 1;
SQLResult: [(Decimal('181'),)]
Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White' LIMIT 1;
> Finished chain.


In [262]:
def extract_answer(qns):
    # Extract the part of the result that starts after "Answer: "
    result = qns['result']
    answer_start = result.find("Answer: ") + len("Answer: ")
    return result[answer_start:].strip()  # Remove leading/trailing whitespaces

In [267]:
few_shots = [
    {'Question': "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer': extract_answer(qns1)},  # Only the answer part from qns1
    
    # {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
    #  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
    #  'SQLResult': "Result of the SQL query",
    #  'Answer': extract_answer(qns2)},  # Only the answer part from qns2
    
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?",
     'SQLQuery': """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': extract_answer(qns3)},  # Only the answer part from qns3
    
    {'Question': "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?",
     'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
     'SQLResult': "Result of the SQL query",
     'Answer': extract_answer(qns4)},  # Only the answer part from qns4
    
    # {'Question': "How many white color Levi's shirt I have?",
    #  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
    #  'SQLResult': "Result of the SQL query",
    #  'Answer': extract_answer(qns5)}  # Only the answer part from qns5
]

In [268]:
print(few_shots[0])

{'Question': 'How many t-shirts do we have left for Nike in XS size and white color?', 'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'", 'SQLResult': 'Result of the SQL query', 'Answer': 'We have 0 t-shirts left for Nike in extra small size and white color.'}


In [31]:
#pip install -U langchain-huggingface

In [271]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L3-v2")
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

In [152]:
#e = embeddings.embed_query("How many white color Levi's shirt I have?")

In [153]:
#len(e)

384

In [272]:
to_vectorize = [" ".join(map(str, example.values())) for example in few_shots]

In [275]:
to_vectorize[2]

"If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' Result of the SQL query 21723"

In [276]:
print(len(to_vectorize))

3


In [277]:
from langchain.vectorstores import Chroma

In [278]:
processed_metadata = [
    {key: (str(value) if isinstance(value, dict) else value) for key, value in example.items()}
    for example in few_shots
]

In [282]:
print(len(processed_metadata))

3


In [ ]:
#pip install chromadb==0.5.3

In [ ]:
#pip show chromadb

In [283]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [284]:
from langchain.prompts import SemanticSimilarityExampleSelector

In [285]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

In [286]:
example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': 'We have 0 t-shirts left for Nike in extra small size and white color.',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': 'We have no Nike t-shirts in extra small size and white color.',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'}]

In [287]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [189]:
#from langchain.chains.sql_database.prompt import _mysql_prompt

In [288]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [191]:
# PROMPT_SUFFIX = """
# Only use the following tables:
# {table_info}

# Question: {query}
# """

In [289]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [290]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [291]:
from langchain.prompts import FewShotPromptTemplate

In [292]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],
)

In [293]:
print(few_shot_prompt.input_variables)

['input', 'table_info', 'top_k']


In [344]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [345]:
print(new_chain.input_keys)

['query']


In [ ]:
new_chain.invoke("How much is the price of the inventory for all small size t-shirts?")

In [ ]:
new_chain.invoke("How much is the price of the inventory for all small size t-shirts?")

In [314]:
query2 = "How much is the price of the inventory for all small size t-shirts?"

In [340]:
few_shot_prompt.prefix = few_shot_prompt.prefix.replace("{query}", "{input}")

In [341]:
few_shot_prompt.suffix = few_shot_prompt.suffix.replace("{query}", "{input}")

In [342]:
few_shot_prompt.example_prompt = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],  # Adjust variables
    template=few_shot_prompt.example_prompt.template.replace("{query}", "{input}")
)

In [343]:
print(few_shot_prompt.input_variables)

['input', 'table_info', 'top_k']


In [356]:
print(new_chain.input_keys)

['input']


In [354]:
input = new_chain.invoke(
    {
        "input": "SELECT * FROM Artist LIMIT 10;"
    }
)
print(input)

# Run the validated query on the database
db.run(input)



> Entering new SQLDatabaseChain chain...
SELECT * FROM Artist LIMIT 10;
SQLQuery:

KeyError: "'query'"

In [336]:
input_data = {
    "input": query2,  # String (valid)
    "table_info": str(db.table_info),  # Ensure table_info is a string
    "top_k": str(2)  # Convert integer to string
}

In [337]:
formatted_prompt = few_shot_prompt.format(**input_data)

KeyError: "'query'"

In [328]:
print(few_shot_prompt.input_variables)

['input', 'table_info', 'top_k']
